In [ ]:
# %%capture
!pip install torch --index-url https://download.pytorch.org/whl/cu118

In [ ]:
#Dataset
!wget "https://public.roboflow.com/ds/33zG9JOCMA?key=jZBzXHc048" -O mask-wearing.zip
!unzip mask-wearing.zip -d mask_wearing

In [ ]:
!pip install ultralytics

In [ ]:
# YOLOv5 Setup
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

In [ ]:
# Define dataset path
DATASET_PATH = "/content/mask_dataset/"
# Changed data.yaml relative train, valid paths to absolute paths

# Visualizing some dataset images
image_files = [os.path.join(DATASET_PATH, "train", "images", f) for f in os.listdir(os.path.join(DATASET_PATH, "train", "images"))[5:10]]

fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for img_path, ax in zip(image_files, axes):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img)
    ax.axis("off")
plt.show()


In [ ]:
# Train YOLOv5

# !python train.py --img 640 --batch 16 --epochs 50 --data /content/mask_wearing/data.yaml--weights yolov5s.pt
!python train.py --weights yolov5s.pt --img 640 --batch-size 16 --epochs 50 --data /content/mask_dataset/data.yaml



In [ ]:
!find /content -name "best.pt"


In [ ]:
# Test YOLOv5
# !python detect.py --weights runs/train/exp/weights/best.pt --source {DATASET_PATH}/test/images
!python detect.py --weights /content/yolov5/yolov5/runs/train/exp2/weights/best.pt --source /content/mask_dataset/test/images --conf-thres 0.25 --iou-thres 0.45


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
import random
import os


In [ ]:
# # Force TensorFlow to use CPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# # Clear previous models from memory
# tf.keras.backend.clear_session()

base_model = ResNet50(weights='imagenet', include_top=False)


In [ ]:

print(f"TensorFlow version: {tf.__version__}")
print(f"Is built with CUDA: {tf.test.is_built_with_cuda()}")
print(f"Is GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Define the layers to extract features from
layer_names = ['conv1_relu', 'conv2_block3_out', 'conv4_block6_out']
feature_extractor = Model(inputs=base_model.input, outputs=[base_model.get_layer(name).output for name in layer_names])

# Function to preprocess an image
def load_and_preprocess_img(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # ResNet50 expects 224x224
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


# Function to display feature maps
def display_feature_maps(features, layer_name):
    num_features = features.shape[-1]  # Number of filters
    size = features.shape[1]  # Size of feature map
    cols = 8  # Number of columns in the display grid
    rows = num_features // cols  # Number of rows

    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))
    for i in range(rows):
        for j in range(cols):
            ax = axes[i, j]
            ax.imshow(features[0, :, :, i * cols + j], cmap='viridis')
            ax.axis('off')
    plt.suptitle(f'Feature Maps from {layer_name}', fontsize=16)
    plt.show()


In [ ]:
import kagglehub
import shutil

# Download the dataset (default location)
download_path = kagglehub.dataset_download("jessicali9530/stanford-cars-dataset")

# Define your custom save location
save_path = "/content/dataset/stanford-cars"

# Move the dataset to your desired location
shutil.move(download_path, save_path)

print("Dataset moved to:", save_path)


In [ ]:

dataset_path = "/content/dataset/stanford-cars/cars_test/cars_test/"  # Change this to your dataset folder

# Select 5 random images
image_files = random.sample(os.listdir(dataset_path), 5)
image_paths = [os.path.join(dataset_path, img) for img in image_files]

# Display the images
plt.figure(figsize=(10, 6))  # Set figure size
for i, img_path in enumerate(image_paths):
    img = image.load_img(img_path, target_size=(224, 224))  # Load image correctly
    plt.subplot(1, 5, i + 1)  # Arrange in 1 row, 5 columns
    plt.imshow(img)
    plt.axis('off')  # Hide axis
    plt.title(f"Image {i+1}")

plt.show()  # Show images


In [ ]:

# Process and extract features
for img_path in image_paths:
    img_array = load_and_preprocess_img(img_path)
    features = feature_extractor.predict(img_array)

    # Display original image
    img = image.load_img(img_path, target_size=(224, 224))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Original Image')
    plt.show()

    # Display extracted features
    for feature, layer in zip(features, layer_names):
        display_feature_maps(feature, layer)